#  Extracting Statistical Information from AFM Scans

## Importing Packages

In [28]:
import csv
import math
import numpy as np
from scipy.signal import argrelextrema
from findpeaks import findpeaks as findpeaks
import pandas as pd

## Defining Functions

In [16]:
# Extpected input: Outputted cross sectional data matrix from Pygwy script,
# pass the pixel dimension of your image as the value for n, defaulted to 512 representing a SQUARE 512x512 image.
# Expected Output: Nested list, containing lists of a row-wise breakup of the inital matrix of size n by n.
def SplitDataMatrixIntoRows(data, pixel_dimension_of_image=512):
    nested_list_of_individual_cross_sections = list()
    for i in range(0, len(data), pixel_dimension_of_image):
        nested_list_of_individual_cross_sections.append(data[i:i + pixel_dimension_of_image])
    return np.array(nested_list_of_individual_cross_sections)

# Expected input: Output from SplitDataMatrixIntoRows(),
# pass the physical length of your image in micrometers as "length_in_micrometers" and the same pixel dimension as 
# inputted into SplitDataMatrixIntoRows(), defaulted to length_in_micrometers=6 and pixel_dimension_of_image=512.
# Expected Output: Nested list containing lists of x-axis locations of negative peaks (relative minima)
def ExtractDistancesBetweenRelativeMinima_GrainDiameter(data, length_in_micrometers=6, pixel_dimension_of_image=512):
    scaling_factor = length_in_micrometers/pixel_dimension_of_image
    nested_list_of_xaxis_locations_of_relative_minima_for_each_cross_section = list()
    for each_individual_cross_section in data:
        xaxis_locations_of_relative_minima = argrelextrema(each_individual_cross_section, np.less)
        xaxis_locations_of_relative_minima = xaxis_locations_of_relative_minima[0]
        xaxis_locations_of_relative_minima = xaxis_locations_of_relative_minima*scaling_factor
        nested_list_of_xaxis_locations_of_relative_minima_for_each_cross_section.append(xaxis_locations_of_relative_minima)
    return nested_list_of_xaxis_locations_of_relative_minima_for_each_cross_section
    
# Expected Input: Output from ExtractDistancesBetweenRelativeMinima_GrainDiameter()  
# Expected Output: Two floats: The average grain diameter for all grains in the image and the standard deviation.  
def CalculateAverageAndStandardDeviation_GrainDiameter(data):
    list_of_grain_diameters_for_all_cross_sections = list()
    for each_list_of_relative_minima_locations in data:
        list_of_individual_diameters = np.diff(each_list_of_relative_minima_locations)
        list_of_grain_diameters_for_all_cross_sections.append(list_of_individual_diameters)
    list_of_grain_diameters_for_all_cross_sections = np.concatenate(list_of_grain_diameters_for_all_cross_sections).ravel()
    average_diameter = np.mean(list_of_grain_diameters_for_all_cross_sections)
    standard_deviation_diameter = np.std(list_of_grain_diameters_for_all_cross_sections)
    return average_diameter,standard_deviation_diameter

# Expected Input: Output from SplitDataMatrixIntoRows().
# Expected Output: Nested list containing heights of individual grains.
def ExtractHeightOfRelativeMaxima_GrainHeight(data):
    nested_list_of_grain_heights = list()
    for each_individual_cross_section in data:
        individual_heights = each_individual_cross_section[argrelextrema(each_individual_cross_section, np.greater)[0]]
        nested_list_of_grain_heights.append(individual_heights)
    return nested_list_of_grain_heights

# Expected Input: Output from ExtractHeightOfRelativeMaxima_GrainHeight()
# Expected Output: Two floats: The average grain height for all grains in the image and the standard devaiton.
def CalculateAverageAndStandardDeviation_GrainHeight(data):
    list_of_grain_heights_for_all_cross_sections = np.concatenate(data).ravel()
    average_height = np.mean(list_of_grain_heights_for_all_cross_sections)
    standard_deviation_height = np.std(list_of_grain_heights_for_all_cross_sections)
    return average_height,standard_deviation_height

# Expected Input: Output from ExtractDistancesBetweenRelativeMinima_GrainDiameter()
# Expected Output: Four floats: The average maximum grain diameter from each cross section and the standard deviation.
# The average minimum grain diameter from each cross section and the standard deviation.
def CalculateAverageAndStandardDeviationOfMaximumAndMinimumGrainDiameter(data):
    list_of_maximum_grain_diameter_from_each_cross_section = list()
    list_of_minimum_grain_diameter_from_each_cross_section = list()
    for each_individual_cross_section in data:
        list_of_individual_diameters = np.diff(each_individual_cross_section)
        list_of_maximum_grain_diameter_from_each_cross_section.append(max(list_of_individual_diameters))
        list_of_minimum_grain_diameter_from_each_cross_section.append(min(list_of_individual_diameters))
    average_maximum_grain_diameter = np.mean(list_of_maximum_grain_diameter_from_each_cross_section)
    standard_deviation_of_maximum_grain_diameter = np.std(list_of_maximum_grain_diameter_from_each_cross_section)
    average_minimum_grain_diameter = np.mean(list_of_minimum_grain_diameter_from_each_cross_section)
    standard_deviation_of_minimum_grain_diameter = np.std(list_of_minimum_grain_diameter_from_each_cross_section)
    return average_maximum_grain_diameter, standard_deviation_of_maximum_grain_diameter, average_minimum_grain_diameter, standard_deviation_of_minimum_grain_diameter

# Expected Input: Output from ExtractHeightOfRelativeMaxima_GrainHeight()
# Expected Output: Four floats: The average maximum grain height from each cross section and the standard deviation.
# The average minimum grain height from each cross section and the standard deviation.
def CalculateAverageAndStandardDeviationOfMaximumAndMinimumGrainHeight(data):
    list_of_maximum_grain_heights_from_each_cross_section = list()
    list_of_minimum_grain_heights_from_each_cross_section = list()
    for each_individual_cross_section in data:
        list_of_maximum_grain_heights_from_each_cross_section.append(max(each_individual_cross_section))
        list_of_minimum_grain_heights_from_each_cross_section.append(min(each_individual_cross_section))
    average_maximum_grain_height = np.mean(list_of_maximum_grain_heights_from_each_cross_section)
    standard_deviation_of_maximum_grain_heights = np.std(list_of_maximum_grain_heights_from_each_cross_section)
    average_minimum_grain_height = np.mean(list_of_minimum_grain_heights_from_each_cross_section)
    standard_deviation_of_minimum_grain_heights = np.std(list_of_minimum_grain_heights_from_each_cross_section)
    return average_maximum_grain_height, standard_deviation_of_maximum_grain_heights, average_minimum_grain_height, standard_deviation_of_minimum_grain_heights

## Opening AFM Scan Data (2d arrays)

In [10]:
file_names = ['c11_5','c11_14','c11_31','c11_77','c11_130']

for name in file_names:
    with open(f'../../UR_la_plante/CaCO3 Gwyddion Python/gwyddion_scripts/gwy_data/{name}_full.txt') as f:
        reader = csv.reader(f)
        data = list(reader)
        data = np.array(data[0])
        exec(f'{name}_raw_data = data.astype(float)')

for name in file_names:
    exec(f'{name}_divided = SplitDataMatrixIntoRows({name}_raw_data)')
    
c11_scan_title = 'C11'
c11_all_scans = {'c11_5_':c11_5_divided,
                 'c11_14_':c11_14_divided,
                 'c11_31_':c11_31_divided,
                 'c11_77_':c11_77_divided,
                 'c11_130_':c11_130_divided}
c11_times = np.array([0.000001, 5, 14, 31, 77, 130])

## Adding one more row to our data
This must be done because for some reason the code to extract AFM data omitted the last row of our 2d array. Before adding a row the dimensions were 511x512 and after adding we get a square array of 512x512.

In [11]:
# Before adding row
for key, array in c11_all_scans.items():
    print((key,array.shape))

('c11_5_', (511, 512))
('c11_14_', (511, 512))
('c11_31_', (511, 512))
('c11_77_', (511, 512))
('c11_130_', (511, 512))


In [9]:
for key, array in c11_all_scans.items():
    array = np.vstack([array, array[0]])
    c11_all_scans[key] = array
    print((key,array.shape))

('c11_5_', (512, 512))
('c11_14_', (512, 512))
('c11_31_', (512, 512))
('c11_77_', (512, 512))
('c11_130_', (512, 512))


## Initializing Data Generation Parameters
Utilizing the "Findpeaks" Python package we will make a function that returns statistical information that will be passed in as parameters to out synthetic data generation functions necessary to generate representative data.

**Module Docs for reference:** https://erdogant.github.io/findpeaks/pages/html/index.html

In [46]:
def process_morphology_return_data_only(morphology_title, list_of_scans, times):
    fp = findpeaks(method='topology', whitelist='peak')
    scan_data = dict()
    peak_y_cordinates = list()
    data = {"mu_n_grains":list(),
            "sigma_n_grains":list(),
            "mu_grain_width":list(),
            "sigma_grain_width":list(),
            "mu_grain_amp":list(),
            "sigma_grain_amp":list()}
    
    for index, scan in list_of_scans.items():
        fit = fp.fit(scan)
        list_of_peak_cordinates = fit['groups0']
        for peak in list_of_peak_cordinates:
            if peak[2] != 0.0:
                peak_y_cordinates.append(peak[0][0])
                    
        list_of_crosssections_with_peaks = [scan[i] for i in list(set(peak_y_cordinates))]
        
        scan_mean_n_grains, scan_std_n_grains = len(list_of_peak_cordinates), 1
        
        scan_diams = ExtractDistancesBetweenRelativeMinima_GrainDiameter(list_of_crosssections_with_peaks)
        scan_mean_diameter, scan_std_diameter = CalculateAverageAndStandardDeviation_GrainDiameter(scan_diams)
        
        scan_height = ExtractHeightOfRelativeMaxima_GrainHeight(list_of_crosssections_with_peaks)
        scan_mean_height, scan_std_height = CalculateAverageAndStandardDeviation_GrainHeight(scan_height)
        
        collection = [scan_mean_n_grains, scan_std_n_grains,
                      scan_mean_diameter, scan_std_diameter,
                      scan_mean_height, scan_std_height]
        
        for entry, index in zip(collection, data.keys()): 
            data[index].append(entry)
            
    data['times'] = list(times)

    return data

In [47]:
# Running our data through the function.
c11_original = process_morphology_return_data_only(c11_scan_title, c11_all_scans, c11_times)

[findpeaks] >Finding peaks in 2d-array using topology method..
[findpeaks] >Scaling image between [0-255] and to uint8
[findpeaks] >Conversion to gray image.
[findpeaks] >WARNING: Conversion to gray not possible.
[findpeaks] >Denoising with [fastnl], window: [3].
[findpeaks] >Detect peaks using topology method with limit at None.
[findpeaks] >Fin.
[findpeaks] >Finding peaks in 2d-array using topology method..
[findpeaks] >Scaling image between [0-255] and to uint8
[findpeaks] >Conversion to gray image.
[findpeaks] >WARNING: Conversion to gray not possible.
[findpeaks] >Denoising with [fastnl], window: [3].
[findpeaks] >Detect peaks using topology method with limit at None.
[findpeaks] >Fin.
[findpeaks] >Finding peaks in 2d-array using topology method..
[findpeaks] >Scaling image between [0-255] and to uint8
[findpeaks] >Conversion to gray image.
[findpeaks] >WARNING: Conversion to gray not possible.
[findpeaks] >Denoising with [fastnl], window: [3].
[findpeaks] >Detect peaks using topo

In [48]:
c11_original

{'mu_n_grains': [1389, 881, 805, 688, 665],
 'sigma_n_grains': [1, 1, 1, 1, 1],
 'mu_grain_width': [0.18094496186159628,
  0.21135960715893742,
  0.2517335796659597,
  0.325077236910093,
  0.35365168205177006],
 'sigma_grain_width': [0.09709316122674311,
  0.10388979503856681,
  0.12121153439079689,
  0.1690740178063406,
  0.194093005444742],
 'mu_grain_amp': [44.66049567570916,
  68.1558838765964,
  59.00468987726128,
  42.87113472897744,
  38.16565087655431],
 'sigma_grain_amp': [12.677142810314749,
  11.900218095579584,
  10.790749468725378,
  8.321523550519661,
  7.808176452173206],
 'times': [1e-06, 5.0, 14.0, 31.0, 77.0, 130.0]}

## Export Statistical Values

In [49]:
with open("statistical_data//c11_stats.csv", "w") as outfile:
    writer = csv.writer(outfile)
    writer.writerow(c11_original.keys())
    writer.writerows(zip(*c11_original.values()))

In [50]:
with open('statistical_data/c11_stats.csv') as f:
    df = pd.read_csv(f).T
df

,0,1,2,3,4
mu_n_grains,1389.000000,881.000000,805.000000,688.000000,665.000000
sigma_n_grains,1.000000,1.000000,1.000000,1.000000,1.000000
mu_grain_width,0.180945,0.211360,0.251734,0.325077,0.353652
sigma_grain_width,0.097093,0.103890,0.121212,0.169074,0.194093
mu_grain_amp,44.660496,68.155884,59.004690,42.871135,38.165651
sigma_grain_amp,12.677143,11.900218,10.790749,8.321524,7.808176
times,0.000001,5.000000,14.000000,31.000000,77.000000
